In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import pystorm
from pystorm.hal import HAL
from pystorm.hal.net_builder import NetBuilder
from pystorm.hal.run_control import RunControl

In [2]:
# set parameters for network
#   number of neurons
# create the network
#   set tap points - this can be encapsulated and improved later

# what should I set the gain and bias bits to?
# 
X = 16
Y = 16
NNEURON = X*Y
DIM = 1
FMAX = 1000

hal = pystorm.hal.HAL()

In [3]:
net_builder = NetBuilder(hal)

bad_syn = hal.get_calibration("synapse", "high_bias_magnitude")
SX = X // 2
SY = Y // 2
tap_matrix_syn = net_builder.create_default_yx_taps(SY, SX, DIM, bad_syn)
tap_matrix = net_builder.syn_taps_to_nrn_taps(tap_matrix_syn)
np.savetxt("tap_matrix.txt", tap_matrix)

In [4]:
gain_divs = np.loadtxt("gain_divisors.txt", dtype=int)
biases = np.loadtxt("biases.txt", dtype=int)
net = net_builder.create_single_pool_net(Y, X, tap_matrix, biases=biases, gain_divs=gain_divs)
run_control = RunControl(hal, net)

hal.map(net)

In [7]:
# collect data for training
# calculate decode weights
#   use linear function offset at 0

total_train_time = 3
total_train_points = 11
offset_time = 0.5

train_rates = (FMAX * np.linspace(-1, 1, total_train_points)).reshape((-1, 1))

train_time_ns = (np.linspace(offset_time, total_train_time+offset_time, total_train_points)*1E9).astype(int)
train_time_ns += hal.get_time()

input_obj = net.inputs[0]
input_vals = {input_obj:(train_time_ns, train_rates)}
outputs, spikes = run_control.run_input_sweep(input_vals, get_raw_spikes=True, get_outputs=True)
print(outputs)
print(spikes)

(None, None)
({Pool p1: array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 3, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       [0, 3, 0, ..., 0, 0, 0]], dtype=uint32)}, array([46153410000, 46154410000, 46155410000, ..., 48798410000,
       48799410000, 48800410000], dtype=uint64))


In [6]:
# run tests
#  deliver an input of 0 